In [1]:
# Imports
import pandas as pd
# from sqlalchemy import create_engine
# import sqlalchemy
import os,sys
from pathlib import Path

In [2]:
# Settings
path_to_data = "example_data/example_csv"

In [3]:
# All magic here!

# Create result dict
pandases = {}
# Don't forget to add check for the shema name occupied
pandases["database_schema"] = pd.DataFrame()

database_schema_dict = {}

def get_dataframes_from_csv_folder(pandases_dict , path_to_data , parent_folder = None):
    '''Recurcively read all dataframes to one dict'''
    for file in os.listdir(path_to_data):
        full_path = os.path.join( path_to_data, file)
        if os.path.isfile(full_path):
            # Do stuff with files
            filename_no_extention = file.split('.')[0]
            
            if parent_folder:
                database_schema_dict["parent_folder"] = parent_folder
                database_schema_dict["table_name"] = f"{parent_folder}_{filename_no_extention}"
                dataframe_key = f"{parent_folder}_{filename_no_extention}"
            else:
                database_schema_dict["parent_folder"] = ""
                database_schema_dict["table_name"] = filename_no_extention
                dataframe_key = filename_no_extention
            print(f"Adding: {file} to {filename_no_extention}")
            dataframe = pd.read_csv( full_path , index_col=0 )
            pandases_dict[dataframe_key] = dataframe
            database_schema_dict["colomns"] = list(dataframe.columns)
            # Create schema colomn and push data about this table there.
            series = pd.Series(database_schema_dict)
            pandases["database_schema"][database_schema_dict["table_name"]] = series
            
        elif ( os.path.isdir ( full_path ) ):
            # Do stuff with dirs
            print(f"Dir {file}")
            get_dataframes_from_csv_folder(pandases_dict , full_path , parent_folder = file )
    return pandases_dict

pandases = get_dataframes_from_csv_folder(pandases.copy() , path_to_data)

Dir dynamics_dominants
Adding: Coppel Seleccion.csv to Coppel Seleccion
Adding: Wild & Fork 39s (1).csv to Wild & Fork 39s (1)
Adding: Coppel Moda.csv to Coppel Moda
Adding: Kavak.csv to Kavak
Dir dynamics
Adding: Coppel Seleccion.csv to Coppel Seleccion
Adding: Wild & Fork 39s (1).csv to Wild & Fork 39s (1)
Adding: Coppel Moda.csv to Coppel Moda
Adding: Kavak.csv to Kavak
Adding: emotions_data.csv to emotions_data
Adding: emotions_metrics.csv to emotions_metrics
Adding: statement_irt_result.csv to statement_irt_result


In [4]:
pandases["database_schema"]

,dynamics_dominants_Coppel Seleccion,dynamics_dominants_Wild & Fork 39s (1),dynamics_dominants_Coppel Moda,dynamics_dominants_Kavak,dynamics_Coppel Seleccion,dynamics_Wild & Fork 39s (1),dynamics_Coppel Moda,dynamics_Kavak,emotions_data,emotions_metrics,statement_irt_result
parent_folder,dynamics_dominants,dynamics_dominants,dynamics_dominants,dynamics_dominants,dynamics,dynamics,dynamics,dynamics,,,
table_name,dynamics_dominants_Coppel Seleccion,dynamics_dominants_Wild & Fork 39s (1),dynamics_dominants_Coppel Moda,dynamics_dominants_Kavak,dynamics_Coppel Seleccion,dynamics_Wild & Fork 39s (1),dynamics_Coppel Moda,dynamics_Kavak,emotions_data,emotions_metrics,statement_irt_result
colomns,"[Timestamp, CALM, ANGRY, CONFUSED, SAD, SURPRI...","[Timestamp, CALM, ANGRY, CONFUSED, SAD, SURPRI...","[Timestamp, CALM, ANGRY, CONFUSED, SAD, SURPRI...","[Timestamp, CALM, ANGRY, CONFUSED, SAD, SURPRI...","[Timestamp, ANGRY, CONFUSED, SAD, SURPRISED, D...","[Timestamp, ANGRY, CONFUSED, SAD, SURPRISED, D...","[Timestamp, ANGRY, CONFUSED, SAD, SURPRISED, D...","[Timestamp, ANGRY, CONFUSED, SAD, SURPRISED, D...","[Video, Status, Timestamp, CALM, CONFUSED, FEA...","[stimulus, metrics, value]","[stimulus, message, weight, Coppel Moda.mp4, C..."


In [5]:
# Temp dict to save as row of one database data
database_schema_dict

{'parent_folder': '',
 'table_name': 'statement_irt_result',
 'colomns': ['stimulus',
  'message',
  'weight',
  'Coppel Moda.mp4',
  'Coppel Seleccion.mp4',
  'Kavak.mp4',
  'Wild &amp; Fork 39s (1).mp4']}

In [6]:
pandases.keys()

dict_keys(['database_schema', 'dynamics_dominants_Coppel Seleccion', 'dynamics_dominants_Wild & Fork 39s (1)', 'dynamics_dominants_Coppel Moda', 'dynamics_dominants_Kavak', 'dynamics_Coppel Seleccion', 'dynamics_Wild & Fork 39s (1)', 'dynamics_Coppel Moda', 'dynamics_Kavak', 'emotions_data', 'emotions_metrics', 'statement_irt_result'])

In [7]:
pandases['statement_irt_result']

,stimulus,message,weight,Coppel Moda.mp4,Coppel Seleccion.mp4,Kavak.mp4,Wild &amp; Fork 39s (1).mp4
0,Coppel Moda.mp4,Coppel es mi tienda departmental favorite,52.767086,NaN,NaN,NaN,NaN
1,Coppel Moda.mp4,Coppel tiene la ropa de moda que yo necesito,67.197870,NaN,NaN,NaN,NaN
2,Coppel Moda.mp4,Definitivamente compraré en Coppel,72.293709,NaN,NaN,NaN,NaN
3,Coppel Moda.mp4,Me gusta la publicidad que esta haciendo Coppel,80.062845,NaN,NaN,NaN,NaN
4,Coppel Seleccion.mp4,Coppel es mi tienda departmental favorite,53.424729,NaN,NaN,NaN,NaN
5,Coppel Seleccion.mp4,Coppel tiene la ropa de moda que yo necesito,69.035741,NaN,NaN,NaN,NaN
6,Coppel Seleccion.mp4,Definitivamente compraré en Coppel,74.442635,NaN,NaN,NaN,NaN
7,Coppel Seleccion.mp4,Me gusta la publicidad que esta haciendo Coppel,85.888517,NaN,NaN,NaN,NaN
8,Kavak.mp4,Definitivamente compraré mi siguiente auto en ...,52.745898,NaN,NaN,NaN,NaN
9,Kavak.mp4,Kavak me ofrece la forma más segura de comprar...,59.851210,NaN,NaN,NaN,NaN


In [8]:
temp_df = pd.read_csv("/home/miksolo/work/emonomy_grafana/data/csv/statement_irt_result.csv", index_col=0)
temp_df

,stimulus,message,weight,Coppel Moda.mp4,Coppel Seleccion.mp4,Kavak.mp4,Wild &amp; Fork 39s (1).mp4
0,Coppel Moda.mp4,Coppel es mi tienda departmental favorite,52.767086,NaN,NaN,NaN,NaN
1,Coppel Moda.mp4,Coppel tiene la ropa de moda que yo necesito,67.197870,NaN,NaN,NaN,NaN
2,Coppel Moda.mp4,Definitivamente compraré en Coppel,72.293709,NaN,NaN,NaN,NaN
3,Coppel Moda.mp4,Me gusta la publicidad que esta haciendo Coppel,80.062845,NaN,NaN,NaN,NaN
4,Coppel Seleccion.mp4,Coppel es mi tienda departmental favorite,53.424729,NaN,NaN,NaN,NaN
5,Coppel Seleccion.mp4,Coppel tiene la ropa de moda que yo necesito,69.035741,NaN,NaN,NaN,NaN
6,Coppel Seleccion.mp4,Definitivamente compraré en Coppel,74.442635,NaN,NaN,NaN,NaN
7,Coppel Seleccion.mp4,Me gusta la publicidad que esta haciendo Coppel,85.888517,NaN,NaN,NaN,NaN
8,Kavak.mp4,Definitivamente compraré mi siguiente auto en ...,52.745898,NaN,NaN,NaN,NaN
9,Kavak.mp4,Kavak me ofrece la forma más segura de comprar...,59.851210,NaN,NaN,NaN,NaN


In [9]:
s = pd.Series(database_schema_dict)
s

parent_folder                                                     
table_name                                    statement_irt_result
colomns          [stimulus, message, weight, Coppel Moda.mp4, C...
dtype: object

In [10]:
temp_df = pd.DataFrame()
temp_df["123"] = s

In [11]:
temp_df =  pd.DataFrame.from_dict(database_schema_dict, orient='columns')

In [12]:
temp_df

,parent_folder,table_name,colomns
0,,statement_irt_result,stimulus
1,,statement_irt_result,message
2,,statement_irt_result,weight
3,,statement_irt_result,Coppel Moda.mp4
4,,statement_irt_result,Coppel Seleccion.mp4
5,,statement_irt_result,Kavak.mp4
6,,statement_irt_result,Wild &amp; Fork 39s (1).mp4


In [16]:
# Create engine:
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Get script path
my_path = os.getcwd()

# Get settings from .env
env_folder = f"{my_path}/../.env"
load_dotenv(env_folder)  # take environment variables from .env.

# Make database connection
os.environ["DB_HOST"] = "localhost"
database_string = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(database_string)

'/home/miksolo/work/emonomy_grafana/test_tools'

In [18]:
for dataframe_name in pandases.keys():
    pandases[dataframe_name].to_sql(dataframe_name, engine)